In [1]:
!pip install torchinfo

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
#set workdir
os.chdir("/content/gdrive/My Drive/Projects/MSc-Project-River-Segmentation/MSc-Project/Code")

In [4]:
! ls


config.cfg  dataloader.py  models  __pycache__	Testing.ipynb  Training.ipynb


In [5]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from dataloader import Dataset
from torchinfo import summary
import time
import copy
import pdb
from tqdm import tqdm

In [6]:
#training parameters in neptune format
PARAMS = {
    "input_size": 416,
    "output_size": 416,
    "model": "vgg_unet",
    "learning_rate": 0.0001,
    "batch_size": 8,
    'epochs': 25,
    'patience': 10,
    "train_dataset_size": -1, # set train dataset subset. Useful when neet to 
                              # overtrain model with small amount of images.
                              # -1 -all images from train directories.
    "test_dataset_size": -1,  # set test dataset subset.
                              # -1 -all images from train directories.
    "n_classes": 2,
    'image_preload': False,
}

In [7]:
import configparser
config = configparser.ConfigParser()
config.read("./config.cfg")

['./config.cfg']

In [8]:
#neptune installation and initialization
!pip install neptune-client
import neptune
neptune.init(
             project_qualified_name='moad.s.k/MSc-Project',
             api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1MWE3YmU1ZC1jNWViLTQ1ZjUtYTE4NC1mOTgzYmUxNmI5MDkifQ==' # your credentials

             #project_qualified_name=config["neptune"]["project"],
             #api_token=config["neptune"]["token"],
             )
neptune.create_experiment(params=PARAMS)

     |████████████████████████████████| 252 kB 7.1 MB/s 
     |████████████████████████████████| 829 kB 47.6 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 170 kB 64.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
     |████████████████████████████████| 129 kB 69.6 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.10.4-py2.py3-none-any.whl size=432189 sha256=0d36029ab0ad6b2acb0fe086bbc2a9f8ee4b11d88beb1c4c65f7a07e740dacc6
  Stored in directory: /root/.cache/pip/wheels/b0/9c/47/8f8a6f14aa49ca017a36ad440e83c31c9539e93a14aae138da
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=ba769c966bbe5e6ae0ae0898e1680e9b9804a0961e25c75688fa2601453efcf8
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7

Experiment(MSCPROJ-10)

In [9]:
#os.chdir("/content/gdrive/My Drive/Data/sentinel-river-segmentation-dataset/sentinel-river-segmentation-dataset")

In [10]:
! ls


config.cfg  dataloader.py  models  __pycache__	Testing.ipynb  Training.ipynb


In [11]:
#dataset configuration

import os

dataset_dir = os.path.normpath("/content/gdrive/My Drive/Data/sentinel-river-segmentation-dataset/sentinel-river-segmentation-dataset")
x_train_dir = os.path.join(dataset_dir,"x_train")
y_train_dir = os.path.join(dataset_dir,"y_train")
x_test_dir = os.path.join(dataset_dir,"x_test")
y_test_dir = os.path.join(dataset_dir,"y_test")

train_set = Dataset(x_train_dir, y_train_dir, input_size=PARAMS['input_size'], output_size=PARAMS['output_size'], n_classes=PARAMS["n_classes"], count=PARAMS["train_dataset_size"])
test_set = Dataset(x_test_dir, y_test_dir, input_size=PARAMS['input_size'], output_size=PARAMS['output_size'], n_classes=PARAMS["n_classes"], count=PARAMS["test_dataset_size"])

batch_size = PARAMS['batch_size']
dataloaders = {
    'train': DataLoader(train_set, batch_size=PARAMS['batch_size'], shuffle=True, num_workers=0),
    'val': DataLoader(test_set, batch_size=PARAMS['batch_size'], shuffle=True, num_workers=0)
}

In [12]:
# load images - useful if you want to save some time by preloading images (very time-consuming) when 
# the model is still not fuctional and cant run standard training.
if PARAMS['image_preload']:
    for phase in dataloaders:
        for inputs, labels in tqdm(dataloaders[phase]):
            pass

In [13]:
#model loading
if PARAMS['model'] == "simple":
    from models.simple import Simple
    model = Simple()
elif PARAMS['model'] == "vgg_unet":
    from models.vgg_unet import VggUnet
    model = VggUnet()
elif PARAMS['model'] == "vgg_unet_ks":
    from models.vgg_unet_ks import VggUnetKs
    model = VggUnetKs()
elif PARAMS['model'] == "unet":
    from models.unet import UNet
    model = UNet()
elif PARAMS['model'] == "vgg_deconvnet":
    from models.vgg16_deconvnet import VggDeconvNet
    model = VggDeconvNet()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [14]:
#model structure preview

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model_stats = summary(model, input_size=(PARAMS['batch_size'], 3, PARAMS['input_size'], PARAMS['input_size']))
for line in str(model_stats).splitlines():
    neptune.log_text('model_summary', line)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [15]:
from collections import defaultdict
import torch.nn.functional as F
SMOOTH = 1e-6
def iou_metric(outputs: torch.Tensor, labels: torch.Tensor):
    outputs = outputs[:,1,:,:]  # BATCH x 1 x H x W => BATCH x H x W
    labels = labels[:,1,:,:]
    intersection = (outputs * labels).sum(2).sum(1)  # Will be zero if Truth=0 or Prediction=0
    union = (outputs + labels).sum(2).sum(1) - intersection  # Will be zzero if both are 0
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    return iou.mean()
    

def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    return loss.mean()

def calc_loss(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy(pred, target)
    pred = torch.round(pred)
    dice = dice_loss(pred, target)
    loss = bce# * bce_weight + dice * (1 - bce_weight)
    iou = iou_metric(pred, target)
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    metrics['iou'] += iou.data.cpu().numpy() * target.size(0)
    return loss

def print_metrics(metrics, epoch_samples, phase):   
    print(epoch_samples) 
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        neptune.log_metric(phase+"_"+k, metrics[k] / epoch_samples) #log
    print("{}: {}".format(phase, ", ".join(outputs)))    



In [17]:
#training loop
def train_model(model, dataloaders, optimizer, device, num_epochs=25, patience=-1):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0
    no_improvement = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])
                    
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)             

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = calc_loss(outputs, labels, metrics)
                    #print(model.encoder[0].weight.grad)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        #pdb.set_trace()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            epoch_accuracy = metrics['iou'] / epoch_samples

            # deep copy the model
            if phase == 'val':
                if epoch_accuracy > best_accuracy:
                    no_improvement = 0
                    print("Val IoU improved by {}. Saving best model.".format(epoch_accuracy-best_accuracy))
                    best_accuracy = epoch_accuracy
                    best_model_wts = copy.deepcopy(model.state_dict())
                else:
                    no_improvement += 1
                    print("No accuracy improvement since {}/{} epochs.".format(no_improvement,patience))
        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        if patience >= 0 and no_improvement > patience:
            break
    print('Best accuracy: {:4f}'.format(best_accuracy))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [18]:
#model training
optimizer_ft = optim.Adam(model.parameters(), lr=PARAMS['learning_rate'])
model = train_model(model, dataloaders, optimizer_ft, device, num_epochs=PARAMS['epochs'], patience=PARAMS['patience'])

  0%|          | 0/302 [00:00<?, ?it/s]

Epoch 0/24
----------
LR 0.0001


100%|██████████| 302/302 [22:12<00:00,  4.41s/it]


2412


  0%|          | 0/72 [00:00<?, ?it/s]

...

570
val: bce: 0.029353, dice: 0.048811, loss: 0.029353, iou: 0.849002
No accuracy improvement since 2/10 epochs.
7m 25s
Epoch 24/24
----------
LR 0.0001


  0%|          | 0/72 [00:00<?, ?it/s]

2412
train: bce: 0.019980, dice: 0.041202, loss: 0.019980, iou: 0.869614


100%|██████████| 72/72 [00:45<00:00,  1.57it/s]

570
val: bce: 0.042481, dice: 0.060358, loss: 0.042481, iou: 0.817443
No accuracy improvement since 3/10 epochs.
7m 25s
Best accuracy: 0.854043


In [19]:
# save weights
torch.save(model.state_dict(),"state_dict.pth")

In [20]:
neptune.log_artifact('state_dict.pth')

In [21]:
# load weights
model.load_state_dict(torch.load("state_dict.pth", map_location="cpu"))

<All keys matched successfully>

In [22]:
# denormalization function
from torchvision import transforms
inv_normalize = transforms.Normalize(
   mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
   std=[1/0.229, 1/0.224, 1/0.225]
)

def reverse_transform(inp):
    print(inp.shape)
    inp = inv_normalize(inp)
    inp = inp.numpy()
    inp = np.swapaxes(inp, 1, 3)
    inp = np.swapaxes(inp, 1, 2)
    inp = np.clip(inp, 0, 1)
    inp = (inp * 255).astype(np.uint8)
    
    return inp
def labels2mask(labels):
    return labels[:,1,:,:]

In [23]:
# helper function to plot input, ground truth and predict images in grid
import matplotlib.pyplot as plt
def plot_side_by_side(rgb,ground_truth,predict):
    assert rgb.shape[0] == ground_truth.shape[0] == predict.shape[0]
    batch_size = rgb.shape[0]
    fig, axs = plt.subplots(batch_size, 3, figsize=(30,50))
    for i in range(batch_size):
        axs[i, 0].imshow(rgb[i])
        axs[i, 1].imshow(ground_truth[i])
        axs[i, 2].imshow(predict[i])

In [24]:
# visualize example segmentation
import math
model.eval()   # Set model to evaluate mode
test_dataset = Dataset(x_test_dir, y_test_dir, input_size=PARAMS['input_size'], output_size=PARAMS['output_size'], n_classes=PARAMS['n_classes'])
test_loader = DataLoader(test_dataset, batch_size=6, shuffle=True, num_workers=0)
inputs, labels = next(iter(test_loader))
inputs = inputs.to(device)
labels = labels.to(device)

labels = labels.data.cpu().numpy()
pred = model(inputs)

#print(pred)
pred = torch.round(pred)
#print(pred.size())

pred = pred.data.cpu().numpy()
inputs = inputs.data.cpu()

# dataloader return normalized input image, so we have to denormalize before viewing
input_images = reverse_transform(inputs)
# target and predict mask are single channel, so squeeze
target_masks = labels2mask(labels)

pred = labels2mask(pred)

# use helper function to plot
plot_side_by_side(input_images, target_masks, pred)

Output hidden; open in https://colab.research.google.com to view.

In [32]:
#evaluate model
test_dataset = Dataset(x_test_dir, y_test_dir, input_size=PARAMS['input_size'], output_size=PARAMS['output_size'], n_classes=PARAMS['n_classes'])
test_loader = DataLoader(test_dataset, batch_size=PARAMS["batch_size"], shuffle=True, num_workers=0)
intersection=0
union=0
for inputs, labels in tqdm(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    labels = labels.data.cpu().numpy()
    pred = model(inputs)
    pred = torch.round(pred)
    pred = pred.data.cpu().numpy()
    target = labels[:,1,:,:]
    predict = pred[:,1,:,:]
    temp = (target * predict).sum()
    intersection+=temp
    union+=((target + predict).sum() - temp)
iou = intersection/union
print("IoU: {}".format(iou))
neptune.log_metric("total_iou",iou)

100%|██████████| 72/72 [00:48<00:00,  1.49it/s]


IoU: 0.8981777113684103


In [ ]:
!git status

In [29]:
!git add .

In [30]:
!git commit -m "updated"
!git config --global user.email "moad.s.k@gmail.com"
!git config --global user.name "MoadSaadKhorchef"

[main ae57093] updated
 2 files changed, 1 insertion(+), 1 deletion(-)
 rewrite Code/Training.ipynb (89%)
 create mode 100644 Code/state_dict.pth


In [31]:
!git push origin main

Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 54.64 MiB | 10.78 MiB/s, done.
Total 5 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
remote: warning: See http://git.io/iEPt8g for more information.
remote: warning: File Code/state_dict.pth is 58.98 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.
To https://github.com/MoadSaadKhorchef/MSc-Project.git
   7b759f1..ae57093  main -> main
